<a href="https://colab.research.google.com/github/PSriram2517/FMML_IAB_23B21A4538/blob/main/FMML_M1L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning terms and metrics

FMML Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

In [ ]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

In [ ]:
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [ ]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

Orignal target values: [4.526 3.585 3.521 ... 0.923 0.847 0.894]
Target values after conversion: [4 3 3 ... 0 0 0]
Input variables shape: (20640, 8)
Output variables shape: (20640,)


The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [ ]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [ ]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [ ]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [ ]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [ ]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

Number of test samples: 4144
Number of train samples: 16496
Percent of test data: 20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

What is the accuracy of our classifiers on the train dataset?

In [ ]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

Training accuracy using nearest neighbour algorithm: 100.0 %
Training accuracy using random classifier:  16.4375808538163 %


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [ ]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.10852713178294 %
Validation accuracy using random classifier: 16.884689922480618 %


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.048257372654156 %


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [ ]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

Test accuracy: 34.91795366795367 %


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

> Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>cross-validation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.

In [ ]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

In [ ]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

Average validation accuracy: 33.58463539517022 %
Test accuracy: 34.91795366795367 %


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


> Exercise: How does the accuracy of the 3 nearest neighbour classifier change with the number of splits? How is it affected by the split size? Compare the results with the 1 nearest neighbour classifier.

1.Yes, averaging validation accuracy across multiple splits (such as in k-fold cross-validation) typically gives more consistent and reliable results than using a single validation split. This is because:

1. **Reduces Variance:** A single train-test split may lead to biased results due to the random selection of training and validation sets. Averaging across multiple splits reduces this variance, leading to more stable and reliable performance estimates.

2. **Better Use of Data:** In methods like k-fold cross-validation, each data point gets to be in the validation set exactly once, which maximizes the use of the entire dataset for both training and validation, providing a more representative measure of model performance.

3. **Minimizes Bias from Overfitting:** Some splits may result in better performance just by chance due to data peculiarities. Averaging across multiple splits helps avoid this problem and prevents overestimating the model’s performance.

In summary, averaging validation accuracy across multiple splits generally gives a more robust estimate of model performance, especially when the dataset is small or has some inherent variability.

2.Yes, averaging the validation accuracy across multiple splits (like in **k-fold cross-validation**) typically gives a more **accurate estimate of test accuracy** compared to using a single train-test split. Here’s why:

1. **Less Overfitting to a Single Split:** A single validation set can result in performance metrics that are overly optimistic or pessimistic based on the peculiarities of that specific split. By averaging across multiple splits, the model's performance is less likely to be biased by any one particular split of the data, leading to a more general estimate of test accuracy.

2. **More Representative of the Data:** In k-fold cross-validation, each data point is used once in the validation set and multiple times in the training set. This ensures that the estimate reflects the model's performance across different portions of the data, which is more representative of how the model would perform on unseen data.

3. **Reduces Sampling Bias:** Single train-test splits may inadvertently cause sampling bias, where the test set is not fully representative of the data distribution. Multiple splits ensure that the evaluation is more balanced, reducing the impact of such bias.

4. **Improves Generalization Estimate:** Since k-fold cross-validation evaluates the model on several subsets of the data, it provides a better estimate of the model's ability to generalize to new, unseen data (which is the goal of test accuracy).

Therefore, averaging validation accuracy across multiple splits is generally considered a more reliable estimate of test accuracy, particularly when working with small or imbalanced datasets.

3.Yes, increasing the number of iterations generally leads to a better estimate in many computational and statistical processes, especially in stochastic methods like Monte Carlo simulations or iterative algorithms like gradient descent. Here's why:

1. **Reduced Variance**: When dealing with stochastic simulations (like Monte Carlo), more iterations allow the algorithm to sample a larger portion of the possible outcomes. This reduces the variance of the estimate, leading to a more accurate approximation of the true value.

2. **Convergence**: In optimization algorithms (e.g., gradient descent), each iteration brings the solution closer to the optimal value. More iterations allow the algorithm to refine its estimate and converge closer to the minimum or maximum of the function being optimized.

3. **Law of Large Numbers**: In probabilistic estimations, such as when estimating a mean or probability, the law of large numbers ensures that as the number of iterations increases, the average of the results will tend to converge to the true expected value.

### Example Scenarios:

- **Monte Carlo Simulation**: If you're estimating the value of π by randomly placing points in a unit square and checking how many fall inside a quarter circle, the more iterations you run, the closer your estimate of π will likely be to the true value.
  
- **Gradient Descent**: In machine learning, each iteration of gradient descent adjusts the model's parameters to reduce the error. More iterations generally help find a more accurate model, though too many can lead to overfitting.

### Diminishing Returns:
However, there can be diminishing returns. Beyond a certain number of iterations, the improvement in accuracy might become marginal, especially if the system has already converged or if noise in the system dominates after a certain point.

Thus, while more iterations tend to yield better estimates, there’s often a trade-off between computational cost and accuracy.

4.Increasing the number of iterations **cannot fully compensate** for a very small training or validation dataset. Here’s why:

### For the **Training Dataset**:
1. **Overfitting Risk**: With a small training dataset, increasing iterations in training a model can lead to overfitting. The model may learn the patterns, noise, or peculiarities specific to the small dataset, rather than generalizable patterns that apply to new data. More iterations make this problem worse, as the model will continue to fit tightly to the limited data points it has.

2. **Lack of Generalization**: If the training data is too small, no amount of iterations can help the model learn patterns that simply aren’t present. The model will struggle to generalize to unseen data because it hasn’t been exposed to a broad enough variety of cases during training.

3. **Bias**: A small dataset can lead to high bias in the model. The dataset may not be representative of the true distribution, so increasing iterations will just make the model more biased to the particular dataset you have, rather than learning an accurate underlying pattern.

### For the **Validation Dataset**:
1. **Unreliable Performance Metrics**: With a very small validation dataset, performance metrics (e.g., accuracy, precision, recall) will have high variance. Even with more iterations, the validation metrics may fluctuate widely due to the small sample size. As a result, the model's performance may appear inconsistent or unstable, and you won’t get a reliable sense of how the model will perform on new, unseen data.

2. **Overfitting on Validation Set**: If you increase the number of iterations and use the same small validation set repeatedly, the model can effectively "learn" the validation set, leading to overfitting on that set. This gives an inflated sense of how well the model is performing, while in reality, it might not generalize well to new data.

### Conclusion:
- **Increasing iterations is not a solution for a small training or validation dataset.**
  - With a small **training dataset**, you'll need to explore other techniques like data augmentation, transfer learning, or synthetic data generation.
  - With a small **validation dataset**, cross-validation (like k-fold cross-validation) can help make better use of limited data for reliable performance estimation.

So, while more iterations can help refine model estimates and potentially lead to better optimization, they cannot make up for the lack of diverse and representative data. Having a sufficient amount of quality data is key to building a robust and generalizable model.